# Kita Pisahkan datanya menjadi 3

Url Source : https://www.kaggle.com/alxmamaev/flowers-recognition

- train (data latih)
- validation (validasi untuk data latih)
- test (data testing untuk menguji model)

In [ ]:
# extract melalui notebook jika diperlukan
# !unzip flowers-recognition.zip

: 

In [ ]:
import os
import pandas as pd

: 

In [ ]:
mypath= './Jasmine/'

: 

In [ ]:
file_name = []
tag = []
full_path = []
for path, subdirs, files in os.walk(mypath):
    for name in files:
        full_path.append(os.path.join(path, name).replace("\\","/")) 
        tag.append(path.split('/')[-1])        
        file_name.append(name)

: 

In [ ]:
# memasukan variabel yang sudah dikumpulkan pada looping di atas menjadi sebuah dataframe agar rapih
df = pd.DataFrame({"path":full_path,'file_name':file_name,"tag":tag})
df.groupby(['tag']).size()

: 

In [ ]:
#cek sample datanya
df.head()

: 

# Train test split

In [ ]:
#load library untuk train test split
from sklearn.model_selection import train_test_split

: 

In [ ]:
#variabel yang digunakan pada pemisahan data ini
X= df['path']
y= df['tag']

: 

In [ ]:
# split dataset awal menjadi data train dan test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20, random_state=300)

: 

In [ ]:
# kemudian data test dibagi menjadi 2 sehingga menjadi data test dan data validation.
X_test, X_val, y_test, y_val = train_test_split(
    X_test, y_test, test_size=0.5, random_state=100)

: 

In [ ]:
# menyatukan kedalam masing-masing dataframe

df_tr = pd.DataFrame({'path':X_train
              ,'tag':y_train
             ,'set':'train'})

df_te = pd.DataFrame({'path':X_test
              ,'tag':y_test
             ,'set':'test'})

df_val = pd.DataFrame({'path':X_val
              ,'tag':y_val
             ,'set':'validation'})

: 

In [ ]:
print('train size', len(df_tr))
print('val size', len(df_te))
print('test size', len(df_val))

: 

In [ ]:
# melihat proporsi pada masing masing set apakah sudah ok atau masih ada yang ingin diubah
df_all = df_tr.append([df_te,df_val]).reset_index(drop=1)

print('===================================================== \n')
print(df_all.groupby(['set','tag']).size(),'\n')

print('===================================================== \n')

#cek sample datanya
df_all.sample(3)

: 

# Merapihkan ke folder set masing-masing

In [ ]:
# menghapus folder dataset jika diperlukan
#!rm -rf dataset/

: 

In [ ]:
import shutil
from tqdm.notebook import tqdm as tq

: 

In [ ]:
datasource_path = "flowers/"
dataset_path = "dataset/"

: 

In [ ]:
for index, row in tq(df_all.iterrows(), ):
    
    #detect filepath
    file_path = row['path']
    if os.path.exists(file_path) == False:
            file_path = os.path.join(datasource_path,row['tag'],row['image'].split('.')[0])            
    
    #make folder destination dirs
    if os.path.exists(os.path.join(dataset_path,row['set'],row['tag'])) == False:
        os.makedirs(os.path.join(dataset_path,row['set'],row['tag']))
    
    #define file dest
    destination_file_name = file_path.split('/')[-1]
    file_dest = os.path.join(dataset_path,row['set'],row['tag'],destination_file_name)
    
    #copy file from source to dest
    if os.path.exists(file_dest) == False:
        shutil.copy2(file_path,file_dest)

: 

# Finish! Dataset siap digunakan 